In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.tools import StructuredTool
from langchain.agents import initialize_agent, AgentType

llm = ChatOpenAI(
    temperature=0.1,
    model="gpt-4o-mini-2024-07-18",
)

def plus(a, b):
    return a + b

agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    tools=[
        StructuredTool.from_function(
            func=plus,
            name="Sum Calculator",
            description="Use this to perform sums of two numbers. This tool take two arguments, both  should be numbers.",
        )
    ]
)

prompt = "Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12"

agent.invoke(prompt)



> Entering new AgentExecutor chain...
Thought: I need to calculate the sum of the provided numbers to find the total cost. 

Action:
```
{
  "action": "Sum Calculator",
  "action_input": {
    "a": 355.39,
    "b": 924.87
  }
}
```

Observation: 1280.26
Thought:I need to continue adding the remaining numbers to the total. The current total is 1280.26, and I will add the next number, 721.20.

Action:
```
{
  "action": "Sum Calculator",
  "action_input": {
    "a": 1280.26,
    "b": 721.20
  }
}
```


Observation: 2001.46
Thought:I need to continue adding the remaining numbers to the total. The current total is 2001.46, and I will add the next number, 1940.29.

Action:
```
{
  "action": "Sum Calculator",
  "action_input": {
    "a": 2001.46,
    "b": 1940.29
  }
}
```

Observation: 3941.75
Thought:I need to continue adding the remaining numbers to the total. The current total is 3941.75, and I will add the next number, 573.63.

Action:
```
{
  "action": "Sum Calculator",
  "action_inpu

{'input': 'Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12',
 'output': 'The total cost is $5273.38.'}

In [ ]:
# Zero-shot ReAct Agent
# 하나의 변수만을 받을 수 있음
# agent의 오류를 해결하기 위해 handle_parsing_errors=True 세팅
from langchain.chat_models import ChatOpenAI
from langchain.tools import StructuredTool, Tool
from langchain.agents import initialize_agent, AgentType

llm = ChatOpenAI(temperature=0.1)


def plus(inputs):
    a, b = inputs.split(",")
    return float(a) + float(b)


agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    tools=[
        Tool.from_function(
            func=plus,
            name="Sum Calculator",
            description="Use this to perform sums of two numbers. Use this tool by sending a pair of number separated by a comma.\nExample:1,2",
        ),
    ],
)

prompt = "Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12"

agent.invoke(prompt)

In [ ]:
# OpenAI Functions Agent
# 이름에서 알 수 있듯 OpenAI 만 가능
# 프롬프트가 적어져서 비용적인 측면에서 유리
# agent의 행위를 일일이 지정해주지 않아도 되기 때문에 편리함
from typing import Any, Type
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from langchain.agents import initialize_agent, AgentType

llm = ChatOpenAI(temperature=0.1)


class CalculatorToolArgsSchema(BaseModel):
    a: float = Field(description="The first number")
    b: float = Field(description="The second number")


class CalculatorTool(BaseTool):
    name = "CalculatorTool"
    description = """
    Use this to perform sums of two numbers.
    The first and second arguments should be numbers.
    Only receives two arguments.
    """
    args_schema: Type[CalculatorToolArgsSchema] = CalculatorToolArgsSchema

    def _run(self, a, b):
        return a + b


agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors=True,
    tools=[
        CalculatorTool(),
    ],
)

prompt = "Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12"

agent.invoke(prompt)

In [1]:
from typing import Any, Type
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from langchain.agents import initialize_agent, AgentType
from langchain.tools import DuckDuckGoSearchResults

llm = ChatOpenAI(temperature=0.1)


class StockMarketSymbolSearchToolArgsSchema(BaseModel):
    query: str = Field(
        description="The query you will search for.Example query: Stock Market Symbol for Apple Company"
    )


class StockMarketSymbolSearchTool(BaseTool):
    name = "StockMarketSymbolSearchTool"
    description = """
    Use this tool to find the stock market symbol for a company.
    It takes a query as an argument.
    
    """
    args_schema: Type[
        StockMarketSymbolSearchToolArgsSchema
    ] = StockMarketSymbolSearchToolArgsSchema

    def _run(self, query):
        ddg = DuckDuckGoSearchResults()
        return ddg.run(query)


agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors=True,
    tools=[
        StockMarketSymbolSearchTool(),
    ],
)

prompt = "Give me information on Cloudflare's stock and help me analyze if it's a potential good investment. Also tell me what symbol does the stock have."

agent.invoke(prompt)



> Entering new AgentExecutor chain...

Invoking: `StockMarketSymbolSearchTool` with `{'query': 'Cloudflare stock'}`


[snippet: Get the latest Cloudflare Inc stock quote, financials, news, and analysis. See how the company performs in the market and compare it to other competitors in the cloud security sector., title: Cloudflare Inc (NET) Stock Price & News - Google Finance, link: https://www.google.com/finance/quote/NET:NYSE], [snippet: Shares of internet security and content delivery network Cloudflare (NYSE:NET) jumped 5.6% in the afternoon session after the Bureau of Labor Statistics reported that the US producer price index ..., title: Why Is Cloudflare (NET) Stock Soaring Today - Yahoo Finance, link: https://finance.yahoo.com/news/why-cloudflare-net-stock-soaring-165024196.html], [snippet: Get the latest information on Cloudflare Inc (NET), a software company that offers security and web performance solutions. See its stock price, valuation, earnings, news, competitors, and mor

{'input': "Give me information on Cloudflare's stock and help me analyze if it's a potential good investment. Also tell me what symbol does the stock have.",
 'output': "Here is some information on Cloudflare's stock analysis:\n\n1. Cloudflare Inc. (NYSE: NET) reported $0.20 EPS for the quarter, exceeding analysts' consensus estimates of $0.14 by $0.06. The company's revenue for the quarter was up 30.0% compared to the same quarter last year.\n\n2. The stock price has increased by +35.38% in the last 52 weeks. Cloudflare's price volatility, indicated by a beta of 1.10, has been higher than the market average.\n\n3. Analysts have provided a 12-month price forecast for Cloudflare stock, with an average target of 92.82. The low estimate is 57, and the high estimate is 135. The average target predicts an increase of 12.67% from the current stock price of 82.38.\n\nBased on this information, Cloudflare has shown strong financial performance and growth potential. The stock price has been on 

In [ ]:
import os
import requests
from typing import Type
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from langchain.agents import initialize_agent, AgentType
from langchain.utilities import DuckDuckGoSearchAPIWrapper

llm = ChatOpenAI(temperature=0.1, model_name="gpt-3.5-turbo-1106")

alpha_vantage_api_key = os.environ.get("ALPHA_VANTAGE_API_KEY")


class StockMarketSymbolSearchToolArgsSchema(BaseModel):
    query: str = Field(
        description="The query you will search for.Example query: Stock Market Symbol for Apple Company"
    )


class StockMarketSymbolSearchTool(BaseTool):
    name = "StockMarketSymbolSearchTool"
    description = """
    Use this tool to find the stock market symbol for a company.
    It takes a query as an argument.
    
    """
    args_schema: Type[
        StockMarketSymbolSearchToolArgsSchema
    ] = StockMarketSymbolSearchToolArgsSchema

    def _run(self, query):
        ddg = DuckDuckGoSearchAPIWrapper()
        return ddg.run(query)


class CompanyOverviewArgsSchema(BaseModel):
    symbol: str = Field(
        description="Stock symbol of the company.Example: AAPL,TSLA",
    )


class CompanyOverviewTool(BaseTool):
    name = "CompanyOverview"
    description = """
    Use this to get an overview of the financials of the company.
    You should enter a stock symbol.
    """
    args_schema: Type[CompanyOverviewArgsSchema] = CompanyOverviewArgsSchema

    def _run(self, symbol):
        r = requests.get(
            f"https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={alpha_vantage_api_key}"
        )
        return r.json()


class CompanyIncomeStatementTool(BaseTool):
    name = "CompanyIncomeStatement"
    description = """
    Use this to get the income statement of a company.
    You should enter a stock symbol.
    """
    args_schema: Type[CompanyOverviewArgsSchema] = CompanyOverviewArgsSchema

    def _run(self, symbol):
        r = requests.get(
            f"https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol={symbol}&apikey={alpha_vantage_api_key}"
        )
        return r.json()["annualReports"]


class CompanyStockPerformanceTool(BaseTool):
    name = "CompanyStockPerformance"
    description = """
    Use this to get the weekly performance of a company stock.
    You should enter a stock symbol.
    """
    args_schema: Type[CompanyOverviewArgsSchema] = CompanyOverviewArgsSchema

    def _run(self, symbol):
        r = requests.get(
            f"https://www.alphavantage.co/query?function=TIME_SERIES_WEEKLY&symbol={symbol}&apikey={alpha_vantage_api_key}"
        )
        return r.json()


agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors=True,
    tools=[
        StockMarketSymbolSearchTool(),
        CompanyOverviewTool(),
        CompanyIncomeStatementTool(),
        CompanyStockPerformanceTool(),
    ],
)

prompt = "Give me financial information on Cloudflare's stock, considering its financials, income statements and stock performance help me analyze if it's a potential good investment."

agent.invoke(prompt)

In [2]:
from langchain.agents import create_sql_agent, AgentType
from langchain.chat_models import ChatOpenAI
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase

llm = ChatOpenAI(
    temperature=0.1,
    model_name="gpt-4o-mini-2024-07-18",
)
db = SQLDatabase.from_uri("sqlite:///movies.sqlite")
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

agent = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    agent_type=AgentType.OPENAI_FUNCTIONS,
    verbose=True,
)

agent.invoke(
    "Give me the movies that have the highest votes but the lowest budgets and give me the name of their directors also include their gross revenue and votes."
)



> Entering new AgentExecutor chain...

Invoking: `sql_db_list_tables` with ``


directors, movies
Invoking: `sql_db_schema` with `movies`



CREATE TABLE movies (
	id INTEGER, 
	original_title VARCHAR, 
	budget INTEGER, 
	popularity INTEGER, 
	release_date TEXT, 
	revenue INTEGER, 
	title TEXT, 
	vote_average REAL, 
	vote_count INTEGER, 
	overview TEXT, 
	tagline TEXT, 
	uid INTEGER, 
	director_id INTEGER DEFAULT 0 NOT NULL, 
	PRIMARY KEY (id)
)

/*
3 rows from movies table:
id	original_title	budget	popularity	release_date	revenue	title	vote_average	vote_count	overview	tagline	uid	director_id
43597	Avatar	237000000	150	2009-12-10	2787965087	Avatar	7.2	11800	In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but 	Enter the World of Pandora.	19995	4762
43598	Pirates of the Caribbean: At World's End	300000000	139	2007-05-19	961000000	Pirates of the Caribbean: At World's End	6.9	4500	Captain Barbossa, long believed to be dead, has come back to

{'input': 'Give me the movies that have the highest votes but the lowest budgets and give me the name of their directors also include their gross revenue and votes.',
 'output': 'Here are the movies with the highest votes but the lowest budgets, along with their directors, gross revenue, and votes:\n\n1. **Inception**\n   - Director: Christopher Nolan\n   - Budget: $160,000,000\n   - Revenue: $825,532,764\n   - Votes: 13,752\n\n2. **The Dark Knight**\n   - Director: Christopher Nolan\n   - Budget: $185,000,000\n   - Revenue: $1,004,558,444\n   - Votes: 12,002\n\n3. **Avatar**\n   - Director: James Cameron\n   - Budget: $237,000,000\n   - Revenue: $2,787,965,087\n   - Votes: 11,800\n\n4. **The Avengers**\n   - Director: Joss Whedon\n   - Budget: $220,000,000\n   - Revenue: $1,519,557,910\n   - Votes: 11,776\n\n5. **Deadpool**\n   - Director: Tim Miller\n   - Budget: $58,000,000\n   - Revenue: $783,112,979\n   - Votes: 10,995\n\n6. **Interstellar**\n   - Director: Christopher Nolan\n   -